# Лабораторная работа 3. Генетические алгоритмы.

## Мельников Евгений 18ПМИ-1, Екатерина Малышева 18ПМИ-2

In [ ]:
import time
from google.colab import drive
import os
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
from math import gcd, floor
from random import randint
import time
from itertools import *
from scipy.optimize import linprog
import copy

import random

In [ ]:
drive.mount('/drive')
os.listdir('/drive/MyDrive/genetic_benchmarks')

Mounted at /drive


['mat_bays29.txt',
 'opt',
 'dis_att48.txt',
 'dis_a280.txt',
 'dis_fl417.txt',
 'dis_ch150.txt']

## Travelling salesman problem  


In [ ]:
# one chromosome is a sequence of cities without the last one (the starting city)
# e. g.: [1, 3, 2, 4] is the following path  1 -> 3 -> 2 -> 4 -> 1 (permutation encoding)

class tsp_genetic:    
  #
  def __init__(self, adj_mat, num_genes, population_size=4, selection_rate=0.5, num_generations=50, mutation_probability = 0.01, verbose=False):
    assert population_size % 4 == 0, "Population size must be divisible by 4"
    
    self.adj_mat = adj_mat
    self.num_genes = num_genes # number of genes in a chromosome
    self.population_size = population_size # number of chromosomes in a population
    self.selection_rate = selection_rate  # not used
    self.num_generations = num_generations
    self.mutation_probability = mutation_probability
    self.verbose = verbose
    self.selection_probabilities = []  # probability of selection for every chromosome 
    self.population = [] # list of chromosomes of current population
    self.population_fitness = [] # fitness function values for every chromosome
    self.best_chromosome = None 
    self.best_chromosome_fitness = 1000000000
    
    
    for i in range(population_size):
      new_chromosome = self.generate_random_chromosome()
      self.population.append(new_chromosome)
      self.population_fitness.append(self.fitness(new_chromosome))


  def generate_random_chromosome(self): # the length of the output chromosome
    random_numbers = []
    for i in range(len(self.adj_mat)):
      random_numbers.append(random.random())    

    new_chromosome = np.argsort(random_numbers)

    return new_chromosome

  
  def fitness(self, chromosome):
    path_cost = 0
    for gene in range(len(chromosome) - 1):
      starting_vert = chromosome[gene]
      ending_vert = chromosome[gene + 1]      
      path_cost += self.adj_mat[starting_vert][ending_vert]

    starting_vert = chromosome[len(chromosome) - 1]
    ending_vert = chromosome[0]
    path_cost += self.adj_mat[starting_vert][ending_vert]
    
    if path_cost < self.best_chromosome_fitness:  # remembering the best chromosome
      self.best_chromosome_fitness = path_cost
      self.best_chromosome = chromosome.copy()

    return path_cost


  def calc_roulette_wheel_probabilities(self):
    fitness_sum = sum(self.population_fitness)  # sum of all fitness function values

    probabilities = []
    for chromosome_ind in range(self.population_size):
      probabilities.append(self.population_fitness[chromosome_ind] / fitness_sum)  # calculating probabilities

    # here comes the weird and probably quite slow part    
    # we are reversing the list so that the chromosomes with the smaller fitness function
    # have higher chance to be selected

    indeces = np.argsort(probabilities)
    probabilities.sort()
    probabilities.reverse()

    self.selection_probabilities = list(range(0, len(probabilities)))
    for i in range(len(indeces)):
      self.selection_probabilities[indeces[i]] = probabilities[i]

  
  def roulette_wheel_selection(self):
    selected_chromosomes = []
    for i in range(2):  # choosing two parents
      chromosome_index = np.random.choice(range(self.population_size), p=self.selection_probabilities)
      selected_chromosomes.append(self.population[chromosome_index])

    return selected_chromosomes


  def rank_selection(self):
    # sorting by fitness
    indeces_sorted = np.argsort(self.population_fitness)[::-1]  # [::-1] - descending order
    #sorted_population = np.take_along_axis(np.array(self.population), indeces_sorted, axis=0)
    sorted_population = []

    for index in indeces_sorted:
      sorted_population.append(self.population[index])

    selected = []
    for i in range(2):
      # choosing a group
      group_number = np.random.choice(range(0, 4), p=[0.5, 0.3, 0.15, 0.05]) 
      group_size = self.population_size // 4
      offset = group_number*group_size
      
      # random index in a group
      ind_in_a_group = random.randint(0, group_size - 1)  # index in a group
      selected.append(sorted_population[offset + ind_in_a_group]) 
    return selected
    

  def ordered_crossover(self, parents):
    # 1|0 3|2
    # 0|1 2|3
    # the parts |0 3| and |1 2| are called dividers (since I can't come up with a better name)

    divider_length = random.randrange(1, (self.num_genes // 2))
    #divider_length = 2  # for debugging

    left_divider = random.randrange(0, self.num_genes - divider_length)  # index of the first element of a divider
    #left_divider = 1 

    right_divider = left_divider + divider_length - 1  # index of the last element of a divider

    # "give birth" to two empty offsprings    
    offsprings = []
    offsprings.append(list(map(int, np.zeros(self.num_genes))))
    offsprings.append(list(map(int, np.zeros(self.num_genes))))

    # remember the dividers and fill up the offsprings (indeces in dividers correspond to indeces in offsprings)
    dividers = [[], []]
    for i in range(left_divider,  right_divider + 1): 
      #print(i)
      dividers[0].append(parents[0][i])
      offsprings[0][i] = parents[0][i]
      dividers[1].append(parents[1][i])
      offsprings[1][i] = parents[1][i]
    #print(offsprings)

    for parent_ind in range(2):
      
      offspring_ind = 1^parent_ind  # sets offspring_ind to 0 when parent_ind is 1 and vice versa
      parent_gene_ind = right_divider + 1  # this index is used to iterate over the parent's gene, starting after the divider

      for offspring_gene_ind in range(right_divider + 1, self.num_genes):  # iterate over offspring's genes outside the divider
        while True:  # we are looking for an element to insert into an offspring
          if parent_gene_ind >= self.num_genes:  # if parent's index reached the end
            parent_gene_ind = 0

          #print(parent_ind, parent_gene_ind, self.num_genes)
          if parents[parent_ind][parent_gene_ind] not in dividers[offspring_ind]:  
            offsprings[offspring_ind][offspring_gene_ind] = parents[parent_ind][parent_gene_ind]
            parent_gene_ind += 1 
            break
          parent_gene_ind += 1          

      for offspring_gene_ind in range(0, left_divider):
        while True:
          if parent_gene_ind >= self.num_genes:  # if parent's index reached the end
            parent_gene_ind = 0

          if parents[parent_ind][parent_gene_ind] not in dividers[offspring_ind]:
            offsprings[offspring_ind][offspring_gene_ind] = parents[parent_ind][parent_gene_ind]
            parent_gene_ind += 1 
            break
          parent_gene_ind += 1  

      # mutation
      if random.randrange(0, round(1/self.mutation_probability)) == 0:
        self.mutation(offsprings[offspring_ind])
        #print("mutated")

    return offsprings


  def mutation(self, chromosome):
    gene1_ind = random.randrange(0, self.num_genes)
    gene2_ind = random.randrange(0, self.num_genes)
    while gene1_ind == gene2_ind:
      gene2_ind = random.randrange(0, self.num_genes)

    tmp = chromosome[gene1_ind]
    chromosome[gene1_ind] = chromosome[gene2_ind]
    chromosome[gene2_ind] = tmp


  def solve(self):
    start_time = time.time()
    for gen in range(self.num_generations):
      new_population = []
      new_population_fitness = []
      #self.calc_roulette_wheel_probabilities()  # calculating the probabilities for current generation
      for num_selection in range(self.population_size//2):  # since we pick two parents at every selection and produce two offsprings during the crossover
        # selection
        #selected_parents = self.roulette_wheel_selection()
        selected_parents = self.rank_selection()
        # crossover
        offsprings = self.ordered_crossover(selected_parents) # create two offsprings

        # append new offsptring to the new population
        new_population.append(offsprings[0])  
        # and calculate its fitness function
        new_population_fitness.append(self.fitness(offsprings[0]))
        new_population.append(offsprings[1])
        new_population_fitness.append(self.fitness(offsprings[1]))
      
      # copy new population data
      self.population = new_population.copy() 
      self.population_fitness = new_population_fitness.copy()

      if self.verbose:
        print("Generation %s, best fitness is %s" %(gen, self.best_chromosome_fitness)) 

    return time.time() - start_time, self.best_chromosome_fitness, self.best_chromosome

# Benchmarks for tsp

In [ ]:
def fitness(adj_mat, chromosome):
  path_cost = 0

  for gene in range(len(chromosome) - 1):
    starting_vert = chromosome[gene] - 1 
    ending_vert = chromosome[gene + 1] - 1     
    path_cost += adj_mat[starting_vert][ending_vert]

  starting_vert = chromosome[len(chromosome) - 1]
  ending_vert = chromosome[0]
  path_cost += adj_mat[starting_vert][ending_vert]  

  return path_cost


class benchmark_tsp:

  tsp_adj_matrices = {}
  tsp_solutions = {}
  benchmark_names = []
  verbose=False

  def __init__(self, path_tsp, verbose=False): # path to a folder, which contains benchmark files
    self.verbose = verbose
    print("Opening up tsp files...")
    for filename in sorted(os.listdir(path_tsp)):
      if "mat_" in filename:  # graph is represented with an adjacency matrix
        
        self.tsp_adj_matrices[filename[4:]]  = np.loadtxt(path_tsp + '/' + filename)

        if filename[0:-4] + '.opt.txt' in os.listdir(path_tsp  + '/opt/'):
          opt_path = np.loadtxt(path_tsp + '/opt/' + filename[0:-4] + '.opt.txt',  dtype=int)
          fit = fitness(self.tsp_adj_matrices[filename[4:]], opt_path)
          self.tsp_solutions[filename[4:]] = fit
        else:
          self.tsp_solutions[filename[4:]] = 'None'
      
        self.benchmark_names.append(filename[4:])
      elif "dis_" in filename:  # graph is represented with coordinates?
        
        coordinates = np.loadtxt(path_tsp + '/' + filename)
        adj_mat = np.zeros((len(coordinates), len(coordinates)))
        for city_a in range(len(coordinates)):
          for city_b in range(len(coordinates)):
            adj_mat[city_a][city_b] = ((coordinates[city_a][1] - coordinates[city_b][1])**2 + (coordinates[city_a][2] - coordinates[city_b][2])**2)**(1/2)
            adj_mat[city_b][city_a] = adj_mat[city_a][city_b]

        self.tsp_adj_matrices[filename[4:]] = adj_mat           
        if filename[0:-4] + '.opt.txt' in os.listdir(path_tsp  + '/opt/'):
          opt_path = np.loadtxt(path_tsp + '/opt/' + filename[0:-4] + '.opt.txt',  dtype=int)
          fit = fitness(self.tsp_adj_matrices[filename[4:]], opt_path)
          self.tsp_solutions[filename[4:]] = fit
        else:
          self.tsp_solutions[filename[4:]] = 'None'
      
        self.benchmark_names.append(filename[4:])
      

    print("Finished")      


  def check_single_benchmark(self, adj_mat, times_repeat=10): 
    mean_time = 0    
    best_fitness = 1000000 # for tsp
    best_chromosome = None

    for i in range(times_repeat):
      if self.verbose:
        print("Try", i, end='  ---  ')
      tsp_algorithm = tsp_genetic(adj_mat, len(adj_mat), population_size=60, num_generations=100, verbose=False, mutation_probability=0.1)
      solution_data = tsp_algorithm.solve()
      if self.verbose:
        print("Fitness:", solution_data[1])

      mean_time += solution_data[0]
      if solution_data[1] < best_fitness:
        best_fitness = solution_data[1]
        best_chromosome = solution_data[2]

    mean_time = mean_time/times_repeat

    return mean_time, best_fitness, best_chromosome


  def report(self):
    time_table = {}
    fitness_table = {}
    path_table = {}

    for benchmark_name in self.benchmark_names:
      if self.verbose:
        print("Running %s..." %(benchmark_name))

      adj_mat = self.tsp_adj_matrices[benchmark_name]

      results = self.check_single_benchmark(adj_mat)

      time_table[benchmark_name] = [results[0]]
      fitness_table[benchmark_name] = [results[1], self.tsp_solutions[benchmark_name]]
      path_table[benchmark_name] = [results[2]]
      
      # optimal value 

    time_table = pd.DataFrame.from_dict(time_table)
    fitness_table = pd.DataFrame.from_dict(fitness_table)
    fitness_table = fitness_table.rename(index={0: 'Genetic algorithm', 1 : 'Optimal'})
    path_table = pd.DataFrame.from_dict(path_table)

    return time_table, fitness_table, path_table

In [ ]:
bench = benchmark_tsp('/drive/MyDrive/genetic_benchmarks', verbose=True)
time_table, fitness_table, path_table = bench.report()


Opening up tsp files...
Finished
Running a280.txt...
Try 0  ---  Fitness: 32486.875368676956
Try 1  ---  Fitness: 31833.8448595862
Try 2  ---  Fitness: 32035.967115624273
Try 3  ---  Fitness: 31622.34475364379
Try 4  ---  Fitness: 31736.92459006963
Try 5  ---  Fitness: 32163.579229943804
Try 6  ---  Fitness: 31745.40437885154
Try 7  ---  Fitness: 31283.481702756682
Try 8  ---  Fitness: 32167.21753309893
Try 9  ---  Fitness: 30828.882670680905
Running att48.txt...
Try 0  ---  Fitness: 126782.07323383384
Try 1  ---  Fitness: 132771.65151233014
Try 2  ---  Fitness: 127350.21951154384
Try 3  ---  Fitness: 136633.24247117824
Try 4  ---  Fitness: 134478.46914840612
Try 5  ---  Fitness: 138848.04524012015
Try 6  ---  Fitness: 132191.50353553874
Try 7  ---  Fitness: 126752.01163677659
Try 8  ---  Fitness: 126804.93025696599
Try 9  ---  Fitness: 126922.8379076503
Running ch150.txt...
Try 0  ---  Fitness: 49993.373792988896
Try 1  ---  Fitness: 49471.03113592892
Try 2  ---  Fitness: 50221.294668

In [ ]:
time_table

,a280.txt,att48.txt,ch150.txt,fl417.txt,bays29.txt
0,4.928893,1.084098,2.5237,8.308911,0.896804


In [ ]:
fitness_table # понятия не имею, почему такие плохие результаты, вот прям вообще хз

,a280.txt,att48.txt,ch150.txt,fl417.txt,bays29.txt
Genetic algorithm,30828.9,126752.011637,49324.407885,447335,4592.0
Optimal,None,35388.461453,7016.684268,None,2233.0


## KSP



In [ ]:
#начальное решение - генерирование популяции 
def gen_population(backpack_size, pop_size): 
    population=[]
    for i in range(pop_size):
        pop=''
        for j in range(backpack_size):
            pop=pop+str(np.random.choice([0,1]))
        population.append(pop)    
    return population

#декодирование хромосомы

def decode(hrom, backpack_size, weights, capacity, W):
    items=[]
    hrom_weight=0
    hrom_capacity=0
    for i in range(backpack_size):
        if hrom[i]=='1':
            if hrom_weight+weights[i]<=W:
                hrom_weight+=weights[i]
                hrom_capacity+=capacity[i]
                items.append(i)
            else:
                break
    return hrom_capacity, items

#фитнесс функция

def fit_func(population, backpack_size, weights, capacity, W):
    pop_capacity=[]
    pop_items=[]
    for i in range(len(population)):
        hrom_capacity, items = decode(population[i], backpack_size, weights, capacity, W)
        pop_capacity.append(hrom_capacity)
        pop_items.append(items)
    return pop_capacity, pop_items

#новая популяция с помощью колеса фортуны
def rowheel(population, pop_capacity, pop_num):
    fitness_sum=[]
    pop_capacity_sum=sum(pop_capacity)
    fitness=[]
    for i in range(len(pop_capacity)):
        fitness.append((pop_capacity[i]/pop_capacity_sum))
    for i in range(len(population)):
        if i==0:
            fitness_sum.append(fitness[i])
        else:
            fitness_sum.append(fitness_sum[i-1]+fitness[i])

    population_new=[]
    for j in range(pop_num):
        r=np.random.uniform(0,1)
        for i in range(len(fitness_sum)):
            if i==0:
                if r>=0 and r<=fitness_sum[i]:
                    population_new.append(population[i])
            else:
                if r>=fitness_sum[i-1] and r<=fitness_sum[i]:
                    population_new.append(population[i])
    return population_new
#кровосмешение и деторождение
def crossover(population_new, crossprob):
    half=int(len(population_new)/2)
    parents_1=population_new[:half]
    parents_2=population_new[half:]
    np.random.shuffle(parents_1)
    np.random.shuffle(parents_2)
    
    offsprings=[]
    for i in range(half):
        r=np.random.uniform(0,1)
        if r<=crossprob:
            p1=np.random.randint(0,(len(parents_1[i])-1))
            p2=np.random.randint(p1,len(parents_1[i]))
            off_1=parents_1[i][:p1]+parents_2[i][p1:p2]+parents_1[i][p2:]
            off_2=parents_2[i][:p1]+parents_1[i][p1:p2]+parents_2[i][p2:]
        else:
            off_1=parents_1[i]
            off_2=parents_2[i]
        offsprings.append(off_1)
        offsprings.append(off_2)
    
    return offsprings
#мутация
def mutation(offspring, mutprob):
    for i in range(len(offspring)):
        r=np.random.uniform(0,1)
        if r<=mutprob:
            p=np.random.randint(0,len(offspring[i]))
            if p==0:
                if offspring[i][p]=='1':
                    offspring[i]='0'+offspring[i][1:]
                else:
                    offspring[i]='1'+offspring[i][1:]
            else:
                if offspring[i][p]=='1':
                    offspring[i]=offspring[i][:(p-1)]+'0'+offspring[i][p:]
                else:
                    offspring[i]=offspring[i][:(p-1)]+'1'+offspring[i][p:]
            
    return offspring

def gen_knapsack(weights, profits, W, popsize, itr, cross_prob, mut_prob):
    
    start=time.time()
    num_item=len(weights)
    population = gen_population(num_item, popsize)
    pop_capacity, pop_items = fit_func(population, num_item, weights, profits, W)

    best_profit_i=[]
    best_sol_i=[]
    
    for i in range(itr):
        
        pop_capacity, pop_items = fit_func(population, num_item, weights, profits, W)
        population = rowheel(population, pop_capacity, popsize)
        offspring_c = crossover(population, cross_prob)
        offspring_m = mutation(offspring_c, mut_prob)
        mixpopulation = population+offspring_m

        pop_capacity, pop_items = fit_func(mixpopulation, num_item, weights, profits, W)
        population = rowheel(mixpopulation, pop_capacity ,popsize)

        pop_capacity, pop_items = fit_func(population, num_item, weights, profits, W)
        result = pop_capacity

        maxres=max(result)
        maxres_ind=result.index(max(result))

        best_profit_i.append(maxres)
        best_sol_i.append(population[maxres_ind])

    best_profit=max(best_profit_i)
    best_profit_ind=best_profit_i.index(max(best_profit_i))
    total_profit, best_backpack_out = decode(best_sol_i[best_profit_ind],num_item, weights, profits, W)
    return best_backpack_out, total_profit, time.time()-start

## Benchmarks for KSP

In [ ]:
drive.mount('/drive')
os.listdir('/drive/MyDrive/knapsnack_benchmarks')

class benchmark:
  popsize=15
  cross_prob=0.25
  mut_prob=0.2
  itr=1000
  knapsnack_algorithms = [gen_knapsack]
  weights = {}
  profits = {}
  answers = {}
  capacities = {}
  numbers = range(1, 8)

  def __init__(self, path): # path to a folder, which contains benchmark files
    print("Opening up input files...")
    for filename in sorted(os.listdir(path)):
      input = open(path + '/' + filename).read()
      if "_c" in filename:
        self.capacities[filename[2:3]] = int(input)
      elif "_p" in filename:
        self.profits[filename[2:3]] = list(map(int, input.split()))
      elif "_s" in filename:
        answers_onehot = list(map(int, input.split()))
        self.answers[filename[2:3]] = []
        for i in range(len(answers_onehot)):    
          if answers_onehot[i] == 1:            
            self.answers[filename[2:3]].append(i)

        #self.answers[filename[2:3]] = list(map(int, input.split()))
      elif "_w" in filename:
        self.weights[filename[2:3]] = list(map(int, input.split()))
    print("Finished")      


  def check_single_benchmark(self, weights, profits, W, popsize, itr, cross_prob, mut_prob, times_repeat=10): 
    
    mean_time = 0
    for i in range(times_repeat):
        items, resulting_profit, time = gen_knapsack(weights, profits, W, popsize, itr, cross_prob, mut_prob)
        mean_time += time

    mean_time = mean_time/times_repeat

    return items, resulting_profit, mean_time


  def calculate_weight(self, items, weights):
    weight = 0
    for item in items:
      weight += weights[item]
    return weight

  def report(self):

    resulting_profit_table = []
    items_table = []
    times_table = []
    operations_table = []
    weights_table = []

    for algorithm in self.knapsnack_algorithms:
      print("Checking", algorithm.__name__)

      
      items_row = {}
      resulting_profit_row = {}
      times_row = {}
      operations_row = {}
      weights_row = {}

      for num in self.numbers:
        results = self.check_single_benchmark(self.weights[str(num)], self.profits[str(num)], self.capacities[str(num)], self.popsize, self.itr,  self.cross_prob, self.mut_prob )
        
        items_row[str(num)] = results[0]
        resulting_profit_row[str(num)] = results[1]
        times_row[str(num)] = results[2]          
        #operations_row[str(num)]  = results[3]
        weights_row[str(num)] = self.calculate_weight(results[0], self.weights[str(num)])

      resulting_profit_table.append(resulting_profit_row)
      items_table.append(items_row)
      times_table.append(times_row) 
      #operations_table.append(operations_row)
      weights_table.append(weights_row)

    items_table.append(self.answers) # row with optimal selection of weights
              
    resulting_profit_table=pd.DataFrame(resulting_profit_table)
    items_table=pd.DataFrame(items_table)
    times_table=pd.DataFrame(times_table)
    #operations_table=pd.DataFrame(operations_table)
    weights_table=pd.DataFrame(weights_table)
    for i in range(len(self.knapsnack_algorithms)):
      resulting_profit_table.rename(index={i:self.knapsnack_algorithms[i].__name__}, inplace=True)
      items_table.rename(index={i:self.knapsnack_algorithms[i].__name__}, inplace=True)
      times_table.rename(index={i:self.knapsnack_algorithms[i].__name__}, inplace=True)
      #operations_table.rename(index={i:self.knapsnack_algorithms[i].__name__}, inplace=True)
      weights_table.rename(index={i:self.knapsnack_algorithms[i].__name__}, inplace=True)
    items_table.rename(index={len(self.knapsnack_algorithms):"Optimal selection"}, inplace=True) # row with optimal selection of weights

    return resulting_profit_table, items_table, times_table, weights_table

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
bench = benchmark('/drive/MyDrive/knapsnack_benchmarks')
resulting_profit_table, items_table, times_table, weights_table = bench.report()


Opening up input files...
Finished
Checking gen_knapsack


In [ ]:
resulting_profit_table

,1,2,3,4,5,6,7
gen_knapsack,284,51,146,107,895,1735,1449


In [ ]:
times_table

,1,2,3,4,5,6,7
gen_knapsack,0.411652,0.379631,0.38501,0.379236,0.413475,0.399463,0.509806


In [ ]:
weights_table

,1,2,3,4,5,6,7
gen_knapsack,161,26,179,50,102,169,750
